In [62]:
import pandas as pd

import requests
import json
from dotenv import load_dotenv
import os

import sys
sys.path.append("../")
from funciones_data import *

In [ ]:
"""

En este notebook he realizado la exploración y limpieza de los datasets originales:

- Análisis de los datasets
- Camnbio de nombres de columnas
- Eliminación de columnas no relevantes
- Observar nulos y rellenarlos

Una vez que el dataset está preparado;

- Enriquecimiento a través de la API de Google Maps.
- Para ello, he creado una función "coord" que devuelve las coordenadas metiendo la dirección o nombre y la localidad.
- Con las coordenadas obtenidad he creado una nueva columna, coordenadas, donde aplico la función antes creada y me da
todas las coordenadas de todos los puntos de mi dataset.

Finalmente he exportado los dataset en .csv


"""

# Restauración

In [3]:
restt = pd.read_csv('../input/Resultados_DC_es_OK.csv', error_bad_lines=False, sep = ";", encoding='UTF-8')
restt.head()

,COD. INSCRIPCIÓN,NOMBRE,MODALIDAD,CATEGORIA,DIRECCIÓN,LOCALIDAD,MUNICIPIO,ZONA,Especialidad,FECHA DE INSCRIPCION
0,ur000158,101,Restaurantes,Sin definir,Guipuzcoa 27,BERRIOZAR,BERRIOZAR ...,Cuenca de Pamplona,Casera o regional,04/05/1973 0:00:00
1,UR000924,A Queimada,Restaurantes,Segunda/2 Tenedores,"La Barrena, parcela 13 3",TUDELA,TUDELA ...,La Ribera,Casera o regional,05/06/2007 0:00:00
2,UR001069,Acedo,Restaurantes,Segunda/2 Tenedores,Los Arcos 0,ACEDO,MENDAZA ...,Zona Media,Casera o regional,02/12/2014 0:00:00
3,ur000129,Adoquín de la Estafeta,Restaurantes,Sin definir,Calle de la Estafeta 81,PAMPLONA,PAMPLONA <> IRUÑA ...,Cuenca de Pamplona,Casera o regional,13/05/1971 0:00:00
4,UR000389,Aguirre,Restaurantes,Sin definir,"Irún, Km. 7 5",ORICÁIN,EZCABARTE ...,Cuenca de Pamplona,Casera o regional,31/07/1984 0:00:00


In [4]:
restt.shape

(757, 10)

In [5]:
restt.columns

Index(['COD. INSCRIPCIÓN', 'NOMBRE', 'MODALIDAD', 'CATEGORIA', 'DIRECCIÓN',
       'LOCALIDAD', 'MUNICIPIO', 'ZONA', 'Especialidad',
       'FECHA DE INSCRIPCION'],
      dtype='object')

In [6]:
restt.rename(columns={'COD. INSCRIPCIÓN':'CÓDIGO', 'FECHA DE INSCRIPCION':'FECHA', 'Especialidad':'ESPECIALIDAD', 'CATEGORIA':'CATEGORÍA'},inplace=True)
restt.head()

,CÓDIGO,NOMBRE,MODALIDAD,CATEGORÍA,DIRECCIÓN,LOCALIDAD,MUNICIPIO,ZONA,ESPECIALIDAD,FECHA
0,ur000158,101,Restaurantes,Sin definir,Guipuzcoa 27,BERRIOZAR,BERRIOZAR ...,Cuenca de Pamplona,Casera o regional,04/05/1973 0:00:00
1,UR000924,A Queimada,Restaurantes,Segunda/2 Tenedores,"La Barrena, parcela 13 3",TUDELA,TUDELA ...,La Ribera,Casera o regional,05/06/2007 0:00:00
2,UR001069,Acedo,Restaurantes,Segunda/2 Tenedores,Los Arcos 0,ACEDO,MENDAZA ...,Zona Media,Casera o regional,02/12/2014 0:00:00
3,ur000129,Adoquín de la Estafeta,Restaurantes,Sin definir,Calle de la Estafeta 81,PAMPLONA,PAMPLONA <> IRUÑA ...,Cuenca de Pamplona,Casera o regional,13/05/1971 0:00:00
4,UR000389,Aguirre,Restaurantes,Sin definir,"Irún, Km. 7 5",ORICÁIN,EZCABARTE ...,Cuenca de Pamplona,Casera o regional,31/07/1984 0:00:00


In [7]:
restt = restt.drop(['FECHA', 'MUNICIPIO' ], axis = 1)
restt.head()

,CÓDIGO,NOMBRE,MODALIDAD,CATEGORÍA,DIRECCIÓN,LOCALIDAD,ZONA,ESPECIALIDAD
0,ur000158,101,Restaurantes,Sin definir,Guipuzcoa 27,BERRIOZAR,Cuenca de Pamplona,Casera o regional
1,UR000924,A Queimada,Restaurantes,Segunda/2 Tenedores,"La Barrena, parcela 13 3",TUDELA,La Ribera,Casera o regional
2,UR001069,Acedo,Restaurantes,Segunda/2 Tenedores,Los Arcos 0,ACEDO,Zona Media,Casera o regional
3,ur000129,Adoquín de la Estafeta,Restaurantes,Sin definir,Calle de la Estafeta 81,PAMPLONA,Cuenca de Pamplona,Casera o regional
4,UR000389,Aguirre,Restaurantes,Sin definir,"Irún, Km. 7 5",ORICÁIN,Cuenca de Pamplona,Casera o regional


In [8]:
restt['LOCALIDAD'].unique()

array(['BERRIOZAR', 'TUDELA', 'ACEDO', 'PAMPLONA', 'ORICÁIN',
       'GERENDIAIN', 'OLAZTI/OLAZAGUTIA', 'ALTSASU/ALSASUA', 'EULATE',
       'CINTRUÉNIGO', 'ITUREN', 'ZUGARRAMURDI', 'ORORBIA', 'VILLAVA',
       'DONEZTEBE/SANTESTEBAN', 'UHARTE-ARAKIL', 'IMÁRCOAIN', 'BARÁSOAIN',
       'BURLADA', 'LODOSA', 'LEITZA', 'ARANTZA', 'FONTELLAS', 'VALTIERRA',
       'ZUASTI', 'NOÁIN', 'ETXARRI-ARANATZ', 'BELZUNCE', 'LEGASA',
       'AURITZ/BURGUETE', 'LÁCAR', 'TAFALLA', 'MILAGRO', 'MARCALÁIN',
       'SAN MARTÍN DE UNX', 'FALCES', 'LEKUNBERRI', 'CORDOVILLA', 'OLITE',
       'HUARTE', 'CÁRCAR', 'ETXALAR', 'ESTELLA-LIZARRA', 'CORELLA',
       'URROZ-VILLA', 'ESQUÍROZ', 'SANGÜESA', 'MUTILVA', 'BARAÑÁIN',
       'LESAKA', 'VIANA', 'UJUÉ', 'ZULUETA', 'ASTITZ', 'PERALTA',
       'OCHAGAVÍA', 'BERA', 'FUNES', 'ELGORRIAGA', 'BARAIBAR',
       'AURIZBERRI/ESPINAL', 'ARGUEDAS', 'EZKURRA', 'LUZAIDE/VALCARLOS',
       'ARRAITZ-ORKIN', 'ALMANDOZ', 'GORRITI',
       'HIRIBERRI/VILLANUEVA DE AEZKOA', 'IZALZU'

In [9]:
restt = restt.loc[(restt['LOCALIDAD'] == 'PAMPLONA') | (restt['LOCALIDAD'] == 'PUENTE LA REINA') | (restt['LOCALIDAD'] == 'ESTELLA-LIZARRA')]
restt.head()

,CÓDIGO,NOMBRE,MODALIDAD,CATEGORÍA,DIRECCIÓN,LOCALIDAD,ZONA,ESPECIALIDAD
3,ur000129,Adoquín de la Estafeta,Restaurantes,Sin definir,Calle de la Estafeta 81,PAMPLONA,Cuenca de Pamplona,Casera o regional
11,UR000011,Alhambra,Restaurantes,Lujo/ 4 Tenedores,Bergamín 7,PAMPLONA,Cuenca de Pamplona,Casera o regional
12,UR000005,Alsafir,Restaurantes,Segunda/2 Tenedores,Castillo de Maya 39,PAMPLONA,Cuenca de Pamplona,Internacional
15,uc000060,Amabella,Cafeterías,Segunda/1 Taza,Vuelta del Castillo 3,PAMPLONA,Cuenca de Pamplona,NaN
19,UR000013,Amóstegui,Restaurantes,Sin definir,"Pozoblanco, 20-1º 0",PAMPLONA,Cuenca de Pamplona,Casera o regional


In [10]:
restt.shape

(259, 8)

In [11]:
restt['LOCALIDAD'].unique()

array(['PAMPLONA', 'ESTELLA-LIZARRA', 'PUENTE LA REINA'], dtype=object)

In [12]:
restt['LOCALIDAD'] = restt['LOCALIDAD'].apply(lambda x: x.split("-")[0])

In [13]:
restt['LOCALIDAD'].unique()

array(['PAMPLONA', 'ESTELLA', 'PUENTE LA REINA'], dtype=object)

In [14]:
restt.head()

,CÓDIGO,NOMBRE,MODALIDAD,CATEGORÍA,DIRECCIÓN,LOCALIDAD,ZONA,ESPECIALIDAD
3,ur000129,Adoquín de la Estafeta,Restaurantes,Sin definir,Calle de la Estafeta 81,PAMPLONA,Cuenca de Pamplona,Casera o regional
11,UR000011,Alhambra,Restaurantes,Lujo/ 4 Tenedores,Bergamín 7,PAMPLONA,Cuenca de Pamplona,Casera o regional
12,UR000005,Alsafir,Restaurantes,Segunda/2 Tenedores,Castillo de Maya 39,PAMPLONA,Cuenca de Pamplona,Internacional
15,uc000060,Amabella,Cafeterías,Segunda/1 Taza,Vuelta del Castillo 3,PAMPLONA,Cuenca de Pamplona,NaN
19,UR000013,Amóstegui,Restaurantes,Sin definir,"Pozoblanco, 20-1º 0",PAMPLONA,Cuenca de Pamplona,Casera o regional


In [15]:
restt.isnull().sum().sort_values(ascending=False)

ESPECIALIDAD    78
CÓDIGO           0
NOMBRE           0
MODALIDAD        0
CATEGORÍA        0
DIRECCIÓN        0
LOCALIDAD        0
ZONA             0
dtype: int64

In [16]:
restt["ESPECIALIDAD"] = restt["ESPECIALIDAD"].fillna("No definida")

In [17]:
restt.head()

,CÓDIGO,NOMBRE,MODALIDAD,CATEGORÍA,DIRECCIÓN,LOCALIDAD,ZONA,ESPECIALIDAD
3,ur000129,Adoquín de la Estafeta,Restaurantes,Sin definir,Calle de la Estafeta 81,PAMPLONA,Cuenca de Pamplona,Casera o regional
11,UR000011,Alhambra,Restaurantes,Lujo/ 4 Tenedores,Bergamín 7,PAMPLONA,Cuenca de Pamplona,Casera o regional
12,UR000005,Alsafir,Restaurantes,Segunda/2 Tenedores,Castillo de Maya 39,PAMPLONA,Cuenca de Pamplona,Internacional
15,uc000060,Amabella,Cafeterías,Segunda/1 Taza,Vuelta del Castillo 3,PAMPLONA,Cuenca de Pamplona,No definida
19,UR000013,Amóstegui,Restaurantes,Sin definir,"Pozoblanco, 20-1º 0",PAMPLONA,Cuenca de Pamplona,Casera o regional


In [18]:
coord("Calle de la Estafeta 81","Pamplona")

(42.8163744, -1.641321)

In [19]:
coord("Bergamín 7","Pamplona")

(42.8127047, -1.6412544)

In [20]:
coord("Paseo de la chopera 14","Madrid")

(40.3921808, -3.6976416)

In [ ]:
#restt['COORDENADAS'] = restt.apply(lambda x: coord(x['DIRECCIÓN'], x['LOCALIDAD']), axis=1)

In [ ]:
restt.head()

In [ ]:
restt.to_csv("kamino_rest_final.csv", index=False)

In [54]:
restauracion = pd.read_csv('../data/kamino_rest_final.csv', error_bad_lines=False, sep = ",", encoding='UTF-8')
restauracion.head()

,CÓDIGO,NOMBRE,MODALIDAD,CATEGORÍA,DIRECCIÓN,LOCALIDAD,ZONA,ESPECIALIDAD,COORDENADAS
0,ur000129,Adoquín de la Estafeta,Restaurantes,Sin definir,Calle de la Estafeta 81,PAMPLONA,Cuenca de Pamplona,Casera o regional,"(42.8163744, -1.641321)"
1,UR000011,Alhambra,Restaurantes,Lujo/ 4 Tenedores,Bergamín 7,PAMPLONA,Cuenca de Pamplona,Casera o regional,"(42.8127047, -1.6412544)"
2,UR000005,Alsafir,Restaurantes,Segunda/2 Tenedores,Castillo de Maya 39,PAMPLONA,Cuenca de Pamplona,Internacional,"(42.8103933, -1.6367607)"
3,uc000060,Amabella,Cafeterías,Segunda/1 Taza,Vuelta del Castillo 3,PAMPLONA,Cuenca de Pamplona,No definida,"(42.80814669999999, -1.6495808)"
4,UR000013,Amóstegui,Restaurantes,Sin definir,"Pozoblanco, 20-1º 0",PAMPLONA,Cuenca de Pamplona,Casera o regional,"(38.3770629, -4.8494709)"


# Alojamiento

In [21]:
aloj = pd.read_csv('../input/Opendata_Resultados_DD_es.csv', error_bad_lines=False, sep = ";", encoding='UTF-8')
aloj.head()

,Unnamed: 0,COD. INSCRIPCIÓN,NOMBRE,MODALIDAD,CATEGORIA,DIRECCIÓN,LOCALIDAD,MUNICIPIO,ZONA,PLAZAS,FECHA DE INSCRIPCION
0,PAMPLONA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,UAT00704,22 Caldereros,Apartamento Turístico,Segunda/2 Llaves,Calle Caldereria 22 2º,PAMPLONA,PAMPLONA <> IRUÑA ...,Cuenca de Pamplona,3.0,12/12/2017 0:00:00
2,NaN,UH000678,A Pamplona,Hotel,2 Estrellas,Calle Sancho Ramirez 15,PAMPLONA,PAMPLONA <> IRUÑA ...,Cuenca de Pamplona,24.0,19/02/1997 0:00:00
3,NaN,UH000645,Abba Reino de Navarra,Hotel,4 Estrellas,Calle Acella 1,PAMPLONA,PAMPLONA <> IRUÑA ...,Cuenca de Pamplona,176.0,17/01/1996 0:00:00
4,NaN,UHS00856,Abodi II,Hostal,HsG,Calle Sancho Ramírez 31,PAMPLONA,PAMPLONA <> IRUÑA ...,Cuenca de Pamplona,8.0,08/02/2010 0:00:00


In [22]:
aloj.shape

(2375, 11)

In [23]:
aloj.columns

Index(['Unnamed: 0', 'COD. INSCRIPCIÓN', 'NOMBRE', 'MODALIDAD', 'CATEGORIA',
       'DIRECCIÓN', 'LOCALIDAD', 'MUNICIPIO', 'ZONA', 'PLAZAS',
       'FECHA DE INSCRIPCION'],
      dtype='object')

In [24]:
aloj.rename(columns={'COD. INSCRIPCIÓN':'CÓDIGO', 'FECHA DE INSCRIPCION':'FECHA', 'Unnamed: 0':'Índice' },inplace=True)
aloj.head()

,Índice,CÓDIGO,NOMBRE,MODALIDAD,CATEGORIA,DIRECCIÓN,LOCALIDAD,MUNICIPIO,ZONA,PLAZAS,FECHA
0,PAMPLONA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,UAT00704,22 Caldereros,Apartamento Turístico,Segunda/2 Llaves,Calle Caldereria 22 2º,PAMPLONA,PAMPLONA <> IRUÑA ...,Cuenca de Pamplona,3.0,12/12/2017 0:00:00
2,NaN,UH000678,A Pamplona,Hotel,2 Estrellas,Calle Sancho Ramirez 15,PAMPLONA,PAMPLONA <> IRUÑA ...,Cuenca de Pamplona,24.0,19/02/1997 0:00:00
3,NaN,UH000645,Abba Reino de Navarra,Hotel,4 Estrellas,Calle Acella 1,PAMPLONA,PAMPLONA <> IRUÑA ...,Cuenca de Pamplona,176.0,17/01/1996 0:00:00
4,NaN,UHS00856,Abodi II,Hostal,HsG,Calle Sancho Ramírez 31,PAMPLONA,PAMPLONA <> IRUÑA ...,Cuenca de Pamplona,8.0,08/02/2010 0:00:00


In [25]:
aloj = aloj.drop(['FECHA', 'MUNICIPIO', 'Índice'], axis = 1)
aloj.head()

,CÓDIGO,NOMBRE,MODALIDAD,CATEGORIA,DIRECCIÓN,LOCALIDAD,ZONA,PLAZAS
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UAT00704,22 Caldereros,Apartamento Turístico,Segunda/2 Llaves,Calle Caldereria 22 2º,PAMPLONA,Cuenca de Pamplona,3.0
2,UH000678,A Pamplona,Hotel,2 Estrellas,Calle Sancho Ramirez 15,PAMPLONA,Cuenca de Pamplona,24.0
3,UH000645,Abba Reino de Navarra,Hotel,4 Estrellas,Calle Acella 1,PAMPLONA,Cuenca de Pamplona,176.0
4,UHS00856,Abodi II,Hostal,HsG,Calle Sancho Ramírez 31,PAMPLONA,Cuenca de Pamplona,8.0


In [26]:
aloj = aloj.loc[(aloj['LOCALIDAD'] == 'PAMPLONA') | (aloj['LOCALIDAD'] == 'PUENTE LA REINA') | (aloj['LOCALIDAD'] == 'ESTELLA-LIZARRA')]
aloj.head()

,CÓDIGO,NOMBRE,MODALIDAD,CATEGORIA,DIRECCIÓN,LOCALIDAD,ZONA,PLAZAS
1,UAT00704,22 Caldereros,Apartamento Turístico,Segunda/2 Llaves,Calle Caldereria 22 2º,PAMPLONA,Cuenca de Pamplona,3.0
2,UH000678,A Pamplona,Hotel,2 Estrellas,Calle Sancho Ramirez 15,PAMPLONA,Cuenca de Pamplona,24.0
3,UH000645,Abba Reino de Navarra,Hotel,4 Estrellas,Calle Acella 1,PAMPLONA,Cuenca de Pamplona,176.0
4,UHS00856,Abodi II,Hostal,HsG,Calle Sancho Ramírez 31,PAMPLONA,Cuenca de Pamplona,8.0
5,UH000503,AC Ciudad de Pamplona,Hotel,4 Estrellas,Iturrama 21,PAMPLONA,Cuenca de Pamplona,178.0


In [27]:
aloj.shape

(243, 8)

In [28]:
aloj['LOCALIDAD'] = aloj['LOCALIDAD'].apply(lambda x: x.split("-")[0])

In [29]:
aloj["LOCALIDAD"].unique()

array(['PAMPLONA', 'ESTELLA', 'PUENTE LA REINA'], dtype=object)

In [30]:
aloj.head()

,CÓDIGO,NOMBRE,MODALIDAD,CATEGORIA,DIRECCIÓN,LOCALIDAD,ZONA,PLAZAS
1,UAT00704,22 Caldereros,Apartamento Turístico,Segunda/2 Llaves,Calle Caldereria 22 2º,PAMPLONA,Cuenca de Pamplona,3.0
2,UH000678,A Pamplona,Hotel,2 Estrellas,Calle Sancho Ramirez 15,PAMPLONA,Cuenca de Pamplona,24.0
3,UH000645,Abba Reino de Navarra,Hotel,4 Estrellas,Calle Acella 1,PAMPLONA,Cuenca de Pamplona,176.0
4,UHS00856,Abodi II,Hostal,HsG,Calle Sancho Ramírez 31,PAMPLONA,Cuenca de Pamplona,8.0
5,UH000503,AC Ciudad de Pamplona,Hotel,4 Estrellas,Iturrama 21,PAMPLONA,Cuenca de Pamplona,178.0


In [31]:
aloj.isnull().sum().sort_values(ascending=False)

ZONA         2
CÓDIGO       0
NOMBRE       0
MODALIDAD    0
CATEGORIA    0
DIRECCIÓN    0
LOCALIDAD    0
PLAZAS       0
dtype: int64

In [32]:
aloj["ZONA"] = aloj["ZONA"].fillna("no definida")

In [33]:
aloj.head()

,CÓDIGO,NOMBRE,MODALIDAD,CATEGORIA,DIRECCIÓN,LOCALIDAD,ZONA,PLAZAS
1,UAT00704,22 Caldereros,Apartamento Turístico,Segunda/2 Llaves,Calle Caldereria 22 2º,PAMPLONA,Cuenca de Pamplona,3.0
2,UH000678,A Pamplona,Hotel,2 Estrellas,Calle Sancho Ramirez 15,PAMPLONA,Cuenca de Pamplona,24.0
3,UH000645,Abba Reino de Navarra,Hotel,4 Estrellas,Calle Acella 1,PAMPLONA,Cuenca de Pamplona,176.0
4,UHS00856,Abodi II,Hostal,HsG,Calle Sancho Ramírez 31,PAMPLONA,Cuenca de Pamplona,8.0
5,UH000503,AC Ciudad de Pamplona,Hotel,4 Estrellas,Iturrama 21,PAMPLONA,Cuenca de Pamplona,178.0


In [ ]:
#aloj['COORDENADAS'] = aloj.apply(lambda x: coord(x['DIRECCIÓN'], x['LOCALIDAD']), axis=1)

In [34]:
coord("Calle Acella 1", "Pamplona")

(42.8085463, -1.6607907)

In [35]:
coord("Calle Sancho Ramírez 31", "Pamplona")

(42.8082357, -1.6662554)

In [ ]:
#CSV OK!
aloj.to_csv("kamino_aloj_final.csv", index=False)

In [57]:
alojamiento= pd.read_csv('../data/kamino_aloj_final.csv', encoding = 'UTF-8')
alojamiento.head()

,CÓDIGO,NOMBRE,MODALIDAD,CATEGORIA,DIRECCIÓN,LOCALIDAD,ZONA,PLAZAS,COORDENADAS
0,UAT00704,22 Caldereros,Apartamento Turístico,Segunda/2 Llaves,Calle Caldereria 22 2º,PAMPLONA,Cuenca de Pamplona,3.0,"(42.81825260000001, -1.6420324)"
1,UH000678,A Pamplona,Hotel,2 Estrellas,Calle Sancho Ramirez 15,PAMPLONA,Cuenca de Pamplona,24.0,"(42.8072773, -1.6649481)"
2,UH000645,Abba Reino de Navarra,Hotel,4 Estrellas,Calle Acella 1,PAMPLONA,Cuenca de Pamplona,176.0,"(42.8085463, -1.6607907)"
3,UHS00856,Abodi II,Hostal,HsG,Calle Sancho Ramírez 31,PAMPLONA,Cuenca de Pamplona,8.0,"(42.8082357, -1.6662554)"
4,UH000503,AC Ciudad de Pamplona,Hotel,4 Estrellas,Iturrama 21,PAMPLONA,Cuenca de Pamplona,178.0,"(42.8045626, -1.6524894)"


# Patrimonio

In [36]:
patr= pd.read_csv('../input/Opendata_Resultados_PM_es.csv', encoding = 'UTF-8')
patr.head()

,Codrecurso,URLNombreCast,Nombre,URLNombreBuscador,URLIdRecursoCategoria,CodCategoria,IdRecursoCategoria,IDCATEGORIA,NombreLocalidad,DescripZona,Path,ImgFichero,Tipo,ESTILO,Distancia,GEORR_X,GEORR_Y,DIPLOMACOMPROMISO
0,2859,Acueducto de Noáin,Acueducto de Noáin,Patrimonio,NaN,NaN,4256,5020800,NOÁIN,Cuenca de Pamplona,05\02\08\00\,2859FOTOTH1.jpg,Obras de ingeniería,Moderno,NaN,612270,4733591,NO
1,2998,Archivo Real y General de Navarra,Archivo Real y General de Navarra,Patrimonio,NaN,NaN,4257,5020400,PAMPLONA,Cuenca de Pamplona,05\02\04\00\,2998FotoTH1.jpg,Castillos/Palacios,NaN,NaN,610815,4741749,NO
2,2999,Ayuntamiento de Pamplona,Ayuntamiento de Pamplona,Patrimonio,NaN,NaN,4258,5021300,PAMPLONA,Cuenca de Pamplona,05\02\12\00\,2999FotoTH1.jpg,Edificios públicos de interés,"Barroco, Neoclásico",NaN,610832,4741543,NO
3,3307,Barrio de San Pedro de la Rúa,Barrio de San Pedro de la Rúa,Patrimonio,NaN,NaN,4336,5021600,ESTELLA-LIZARRA,Zona Media,05\02\12\00\,3307FotoTH1.jpg,Conjuntos históricos,"Barroco, Gótico, Renacentista, Románico",NaN,579502,4724466,NO
4,3159,Basílica de Nuestra Señora del Puy,Basílica de Nuestra Señora del Puy,Patrimonio,NaN,NaN,4322,5020100,ESTELLA-LIZARRA,Zona Media,05\02\01\00\,3159FotoTH1.jpg,Iglesias y ermitas,Moderno,NaN,579485,4725254,NO


In [37]:
patr.shape

(95, 18)

In [38]:
patr.columns

Index(['Codrecurso', 'URLNombreCast', 'Nombre', 'URLNombreBuscador',
       'URLIdRecursoCategoria', 'CodCategoria', 'IdRecursoCategoria',
       'IDCATEGORIA', 'NombreLocalidad', 'DescripZona', 'Path', 'ImgFichero',
       'Tipo', 'ESTILO', 'Distancia', 'GEORR_X', 'GEORR_Y',
       'DIPLOMACOMPROMISO'],
      dtype='object')

In [39]:
patr = patr.drop(['Codrecurso', 'URLNombreCast','URLIdRecursoCategoria', 'CodCategoria','IdRecursoCategoria', 'Path','ImgFichero','Distancia', 'GEORR_X','GEORR_Y', 'DIPLOMACOMPROMISO', 'URLNombreBuscador'], axis = 1)
patr.head()

,Nombre,IDCATEGORIA,NombreLocalidad,DescripZona,Tipo,ESTILO
0,Acueducto de Noáin,5020800,NOÁIN,Cuenca de Pamplona,Obras de ingeniería,Moderno
1,Archivo Real y General de Navarra,5020400,PAMPLONA,Cuenca de Pamplona,Castillos/Palacios,NaN
2,Ayuntamiento de Pamplona,5021300,PAMPLONA,Cuenca de Pamplona,Edificios públicos de interés,"Barroco, Neoclásico"
3,Barrio de San Pedro de la Rúa,5021600,ESTELLA-LIZARRA,Zona Media,Conjuntos históricos,"Barroco, Gótico, Renacentista, Románico"
4,Basílica de Nuestra Señora del Puy,5020100,ESTELLA-LIZARRA,Zona Media,Iglesias y ermitas,Moderno


In [40]:
patr.rename(columns={'Nombre':'NOMBRE','IDCATEGORIA':'CÓDIGO', 'NombreLocalidad':'LOCALIDAD', 'Tipo':'CATEGORÍA', 'DescripZona' : 'ZONA' },inplace=True)
patr.head()

,NOMBRE,CÓDIGO,LOCALIDAD,ZONA,CATEGORÍA,ESTILO
0,Acueducto de Noáin,5020800,NOÁIN,Cuenca de Pamplona,Obras de ingeniería,Moderno
1,Archivo Real y General de Navarra,5020400,PAMPLONA,Cuenca de Pamplona,Castillos/Palacios,NaN
2,Ayuntamiento de Pamplona,5021300,PAMPLONA,Cuenca de Pamplona,Edificios públicos de interés,"Barroco, Neoclásico"
3,Barrio de San Pedro de la Rúa,5021600,ESTELLA-LIZARRA,Zona Media,Conjuntos históricos,"Barroco, Gótico, Renacentista, Románico"
4,Basílica de Nuestra Señora del Puy,5020100,ESTELLA-LIZARRA,Zona Media,Iglesias y ermitas,Moderno


In [41]:
patr = patr.reindex(columns=['CÓDIGO', 'NOMBRE', 'CATEGORÍA', 'ESTILO', 'LOCALIDAD', 'ZONA'])

In [42]:
patr.head()

,CÓDIGO,NOMBRE,CATEGORÍA,ESTILO,LOCALIDAD,ZONA
0,5020800,Acueducto de Noáin,Obras de ingeniería,Moderno,NOÁIN,Cuenca de Pamplona
1,5020400,Archivo Real y General de Navarra,Castillos/Palacios,NaN,PAMPLONA,Cuenca de Pamplona
2,5021300,Ayuntamiento de Pamplona,Edificios públicos de interés,"Barroco, Neoclásico",PAMPLONA,Cuenca de Pamplona
3,5021600,Barrio de San Pedro de la Rúa,Conjuntos históricos,"Barroco, Gótico, Renacentista, Románico",ESTELLA-LIZARRA,Zona Media
4,5020100,Basílica de Nuestra Señora del Puy,Iglesias y ermitas,Moderno,ESTELLA-LIZARRA,Zona Media


In [43]:
patr.shape

(95, 6)

In [44]:
patr = patr.loc[(patr['LOCALIDAD'] == 'PAMPLONA') | (patr['LOCALIDAD'] == 'PUENTE LA REINA') | (patr['LOCALIDAD'] == 'ESTELLA-LIZARRA')]
patr.head()

,CÓDIGO,NOMBRE,CATEGORÍA,ESTILO,LOCALIDAD,ZONA
1,5020400,Archivo Real y General de Navarra,Castillos/Palacios,NaN,PAMPLONA,Cuenca de Pamplona
2,5021300,Ayuntamiento de Pamplona,Edificios públicos de interés,"Barroco, Neoclásico",PAMPLONA,Cuenca de Pamplona
3,5021600,Barrio de San Pedro de la Rúa,Conjuntos históricos,"Barroco, Gótico, Renacentista, Románico",ESTELLA-LIZARRA,Zona Media
4,5020100,Basílica de Nuestra Señora del Puy,Iglesias y ermitas,Moderno,ESTELLA-LIZARRA,Zona Media
7,5020400,Cámara de Comptos,Castillos/Palacios,Gótico,PAMPLONA,Cuenca de Pamplona


In [45]:
patr.shape

(23, 6)

In [46]:
patr['LOCALIDAD'] = patr['LOCALIDAD'].apply(lambda x: x.split("-")[0])

In [47]:
patr.sample()

,CÓDIGO,NOMBRE,CATEGORÍA,ESTILO,LOCALIDAD,ZONA
47,5020100,Iglesia del Santo Sepulcro,Iglesias y ermitas,"Gótico, Románico",ESTELLA,Zona Media


In [48]:
patr.isnull().sum().sort_values(ascending=False)

ESTILO       3
CÓDIGO       0
NOMBRE       0
CATEGORÍA    0
LOCALIDAD    0
ZONA         0
dtype: int64

In [49]:
patr["ESTILO"] = patr["ESTILO"].fillna("no definido")

In [50]:
patr.head()

,CÓDIGO,NOMBRE,CATEGORÍA,ESTILO,LOCALIDAD,ZONA
1,5020400,Archivo Real y General de Navarra,Castillos/Palacios,no definido,PAMPLONA,Cuenca de Pamplona
2,5021300,Ayuntamiento de Pamplona,Edificios públicos de interés,"Barroco, Neoclásico",PAMPLONA,Cuenca de Pamplona
3,5021600,Barrio de San Pedro de la Rúa,Conjuntos históricos,"Barroco, Gótico, Renacentista, Románico",ESTELLA,Zona Media
4,5020100,Basílica de Nuestra Señora del Puy,Iglesias y ermitas,Moderno,ESTELLA,Zona Media
7,5020400,Cámara de Comptos,Castillos/Palacios,Gótico,PAMPLONA,Cuenca de Pamplona


In [ ]:
#patr['COORDENADAS'] = patr.apply(lambda x: coord(x['NOMBRE'], x['LOCALIDAD']), axis=1)

In [ ]:
#patr.to_csv("kamino_patr_final.csv", index=False)

In [56]:
patrimonio= pd.read_csv('../data/kamino_patr_final.csv', encoding = 'UTF-8')
patrimonio.head()

,CÓDIGO,NOMBRE,CATEGORÍA,ESTILO,LOCALIDAD,ZONA,COORDENADAS
0,5020400,Archivo Real y General de Navarra,Castillos/Palacios,no definido,PAMPLONA,Cuenca de Pamplona,"(42.8201142, -1.6444451)"
1,5021300,Ayuntamiento de Pamplona,Edificios públicos de interés,"Barroco, Neoclásico",PAMPLONA,Cuenca de Pamplona,"(42.818428, -1.644141)"
2,5021600,Barrio de San Pedro de la Rúa,Conjuntos históricos,"Barroco, Gótico, Renacentista, Románico",ESTELLA,Zona Media,"(42.66888549999999, -2.0293285)"
3,5020100,Basílica de Nuestra Señora del Puy,Iglesias y ermitas,Moderno,ESTELLA,Zona Media,"(42.67517530000001, -2.0301336)"
4,5020400,Cámara de Comptos,Castillos/Palacios,Gótico,PAMPLONA,Cuenca de Pamplona,"(42.8174347, -1.6458735)"


In [51]:
coord("Archivo Real y General de Navarra", "Pamplona")

(42.8201142, -1.6444451)

In [52]:
coord("BASÍLICA DE NUESTRA SEÑORA DEL PUY", "PAMPLONA")

(42.67517530000001, -2.0301336)